In [352]:
import jax.numpy as jnp
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification
from datetime import date
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [353]:
# [10 Marks] Download a week of PM2.5 dataset from the OpenAQ website for Delhi. Fill
# in the missing data with appropriate methods. Use i) the sklearn Random Forest
# regressor; ii) sklearn Linear regression and iii) Gaussian process regressor to interpolate
# the PM2.5 values across the space. Use all the available features that you can get from
# the OpenAQ website or elsewhere (e.g., meteorological variables). Compare the results
# and prepare a table showing the metrics in K-Fold cross-validation setting.


In [354]:
import requests

In [355]:
response = requests.get("https://api.openaq.org/v2/measurements?location_id=8118&parameter=pm25&date_from=2023-02-01T00:00:00Z&date_to=2023-02-08T00:00:00Z&limit=1000")

In [356]:
data=response.json()

In [357]:
data
del data['meta']

In [358]:
df= pd.DataFrame(data['results'])

In [359]:
df

,locationId,location,parameter,value,date,unit,coordinates,country,city,isMobile,isAnalysis,entity,sensorType
0,8118,US Diplomatic Post: New Delhi,pm25,74.0,"{'utc': '2023-02-07T23:30:00+00:00', 'local': ...",µg/m³,"{'latitude': 28.63576, 'longitude': 77.22445}",IN,Delhi,False,False,government,reference grade
1,8118,US Diplomatic Post: New Delhi,pm25,55.0,"{'utc': '2023-02-07T22:30:00+00:00', 'local': ...",µg/m³,"{'latitude': 28.63576, 'longitude': 77.22445}",IN,Delhi,False,False,government,reference grade
2,8118,US Diplomatic Post: New Delhi,pm25,54.0,"{'utc': '2023-02-07T21:30:00+00:00', 'local': ...",µg/m³,"{'latitude': 28.63576, 'longitude': 77.22445}",IN,Delhi,False,False,government,reference grade
3,8118,US Diplomatic Post: New Delhi,pm25,58.0,"{'utc': '2023-02-07T20:30:00+00:00', 'local': ...",µg/m³,"{'latitude': 28.63576, 'longitude': 77.22445}",IN,Delhi,False,False,government,reference grade
4,8118,US Diplomatic Post: New Delhi,pm25,51.0,"{'utc': '2023-02-07T19:30:00+00:00', 'local': ...",µg/m³,"{'latitude': 28.63576, 'longitude': 77.22445}",IN,Delhi,False,False,government,reference grade
...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,8118,US Diplomatic Post: New Delhi,pm25,100.0,"{'utc': '2023-02-01T04:30:00+00:00', 'local': ...",µg/m³,"{'latitude': 28.63576, 'longitude': 77.22445}",IN,Delhi,False,False,government,reference grade
164,8118,US Diplomatic Post: New Delhi,pm25,80.0,"{'utc': '2023-02-01T03:30:00+00:00', 'local': ...",µg/m³,"{'latitude': 28.63576, 'longitude': 77.22445}",IN,Delhi,False,False,government,reference grade
165,8118,US Diplomatic Post: New Delhi,pm25,91.0,"{'utc': '2023-02-01T02:30:00+00:00', 'local': ...",µg/m³,"{'latitude': 28.63576, 'longitude': 77.22445}",IN,Delhi,False,False,government,reference grade
166,8118,US Diplomatic Post: New Delhi,pm25,92.0,"{'utc': '2023-02-01T01:30:00+00:00', 'local': ...",µg/m³,"{'latitude': 28.63576, 'longitude': 77.22445}",IN,Delhi,False,False,government,reference grade


In [360]:
df.isnull().sum()

locationId     0
location       0
parameter      0
value          0
date           0
unit           0
coordinates    0
country        0
city           0
isMobile       0
isAnalysis     0
entity         0
sensorType     0
dtype: int64

In [361]:
df.describe()

,locationId,value
count,168.0,168.000000
mean,8118.0,99.339286
std,0.0,42.937753
min,8118.0,20.000000
25%,8118.0,68.750000
50%,8118.0,95.000000
75%,8118.0,120.250000
max,8118.0,235.000000


In [362]:
date= dict(df['date'])

In [363]:
date_local= []
date_utc=[]
for i in range(len(date)):
  date_local.append( date[i]['local'])
  date_utc.append(date[i]['utc'])
local= pd.DataFrame(date_local)
utc= pd.DataFrame(date_utc)

In [364]:
df1=df.drop(['location', 'locationId','country', 'city','coordinates','isAnalysis', 'entity', 'sensorType', 'isMobile'], axis= 1)

In [365]:
df1=df1.drop('date', axis=1)

In [366]:
df1= pd.concat([df1,local, utc], axis= 1)

In [367]:
df1.columns=['parameter', 'value (PM2.5 in ug/m3)', 'unit', 'local','utc' ] 

In [368]:
df1 = df1.drop(['parameter', 'unit'], axis= 1)

In [369]:
df1= df1.drop(['utc'], axis= 1)

In [370]:
df1

,value (PM2.5 in ug/m3),local
0,74.0,2023-02-08T05:00:00+05:30
1,55.0,2023-02-08T04:00:00+05:30
2,54.0,2023-02-08T03:00:00+05:30
3,58.0,2023-02-08T02:00:00+05:30
4,51.0,2023-02-08T01:00:00+05:30
...,...,...
163,100.0,2023-02-01T10:00:00+05:30
164,80.0,2023-02-01T09:00:00+05:30
165,91.0,2023-02-01T08:00:00+05:30
166,92.0,2023-02-01T07:00:00+05:30


In [371]:
date_time = df1['local']
date_time = list(date_time)

In [372]:
time = []
date=[]
for i in range(len(date_time)):
  d_t=date_time[i].split('T')
  date.append(d_t[0])
  time.append(d_t[1])
print(date)
print(time)

['2023-02-08', '2023-02-08', '2023-02-08', '2023-02-08', '2023-02-08', '2023-02-08', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-07', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-06', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023-02-05', '2023

In [373]:
for i in range(len(date)):
 date[i]= date[i].split('-')
 date[i]= date[i][2]
for i in range(len(time)):
 time[i]= time[i].split('+')
 time[i]= time[i][0]

In [374]:
time=list(time)
for i in range(len(time)):
 time[i]= time[i].split(':')
 time[i]= time[i][0]
time

['05',
 '04',
 '03',
 '02',
 '01',
 '00',
 '23',
 '22',
 '21',
 '20',
 '19',
 '18',
 '17',
 '16',
 '15',
 '14',
 '13',
 '12',
 '11',
 '10',
 '09',
 '08',
 '07',
 '06',
 '05',
 '04',
 '03',
 '02',
 '01',
 '00',
 '23',
 '22',
 '21',
 '20',
 '19',
 '18',
 '17',
 '16',
 '15',
 '14',
 '13',
 '12',
 '11',
 '10',
 '09',
 '08',
 '07',
 '06',
 '05',
 '04',
 '03',
 '02',
 '01',
 '00',
 '23',
 '22',
 '21',
 '20',
 '19',
 '18',
 '17',
 '16',
 '15',
 '14',
 '13',
 '12',
 '11',
 '10',
 '09',
 '08',
 '07',
 '06',
 '05',
 '04',
 '03',
 '02',
 '01',
 '00',
 '23',
 '22',
 '21',
 '20',
 '19',
 '18',
 '17',
 '16',
 '15',
 '14',
 '13',
 '12',
 '11',
 '10',
 '09',
 '08',
 '07',
 '06',
 '05',
 '04',
 '03',
 '02',
 '01',
 '00',
 '23',
 '22',
 '21',
 '20',
 '19',
 '18',
 '17',
 '16',
 '15',
 '14',
 '13',
 '12',
 '11',
 '10',
 '09',
 '08',
 '07',
 '06',
 '05',
 '04',
 '03',
 '02',
 '01',
 '00',
 '23',
 '22',
 '21',
 '20',
 '19',
 '18',
 '17',
 '16',
 '15',
 '14',
 '13',
 '12',
 '11',
 '10',
 '09',
 '08',
 '07',

In [375]:
date= pd.DataFrame(date)
time= pd.DataFrame(time)
df1= pd.concat([df1,date,time], axis= 1)

In [376]:
df1.columns=['value (PM2.5 in ug/m3)','local','date (Febuary- 2023)','time']
df1= df1.drop('local', axis= 1)

In [377]:
df1

,value (PM2.5 in ug/m3),date (Febuary- 2023),time
0,74.0,08,05
1,55.0,08,04
2,54.0,08,03
3,58.0,08,02
4,51.0,08,01
...,...,...,...
163,100.0,01,10
164,80.0,01,09
165,91.0,01,08
166,92.0,01,07


In [378]:
df1['date (Febuary- 2023)']=pd.to_numeric(df1['date (Febuary- 2023)'])

In [379]:
x= df1.iloc [:, 1:]
y= df1.iloc [:, 0:1]

In [380]:
x_train, x_test, y_train, y_test= train_test_split(x,y , test_size=0.20, random_state=42)

#**Random** **Regressor**

In [381]:
regr = RandomForestRegressor(max_depth=2, random_state=0)
regr.fit(x_train, y_train)
y_pred_random=regr.predict(x_test)

<ipython-input-381-9a7724be1993>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regr.fit(x_train, y_train)


#**Linear Regression**

In [382]:
from sklearn.linear_model import LinearRegression

In [383]:
reg = LinearRegression().fit(x_train, y_train)
y_pred_linear= reg.predict(x_test)

#**Gaussian Process Regressor**

In [384]:
from sklearn.gaussian_process import GaussianProcessRegressor

In [385]:
gpr = GaussianProcessRegressor().fit(x_train, y_train)
y_pred_gpr=gpr.predict(x_test)

In [388]:
from sklearn.metrics import r2_score,mean_squared_error

In [390]:
print(r2_score(y_test,y_pred_random))
print(r2_score(y_test,y_pred_linear))
print(r2_score(y_test,y_pred_gpr))


0.47044382108995386
0.05032239978211306
0.6686220660039824
